8.1 Triple Step

In [1]:
def triple_step(n):
    if n <= 0:
        raise ValueError("Must be bigger than 0")

    if n == 1:
        return 1
    
    # {1, 1}, {2}
    if n == 2:
        return 2
    
    # {1, 1, 1}, {1, 2}, {2, 1}, 3
    if n == 3:
        return 4
    
    return triple_step(n - 1) + triple_step(n - 2) + triple_step(n - 3)

In [2]:
triple_step(4)

8.2 Robot in a Grid

In [3]:
from typing import NamedTuple


class Direction(NamedTuple):
    dx: int
    dy: int

In [4]:
RIGHT = Direction(0, 1)
DOWN = Direction(1, 0)

In [5]:
from collections import deque


class Board:
    def __init__(self, blocks):
        self.blocks = blocks
        self.trace = [[None for column in row] for row in blocks]
    
    def get_start(self):
        return 0, 0
    
    def get_destination(self):
        return len(self.blocks) - 1, len(self.blocks[0]) - 1
    
    def is_start(self, i, j):
        return (i, j) == self.get_start()
    
    def is_destination(self, i, j):
        return (i, j) == self.get_destination()
    
    def is_blocked(self, i, j):
        return self.blocks[i][j] == 1
    
    def is_in_boards(self, i, j):
        return 0 <= i < len(self.blocks) and 0 <= j < len(self.blocks[0])
    
    def neighbors(self, i, j):
        if self.is_in_boards(i + RIGHT.dx, j + RIGHT.dy):
            yield (i + RIGHT.dx, j + RIGHT.dy)
        
        if self.is_in_boards(i + DOWN.dx, j + DOWN.dy):
            yield (i + DOWN.dx, j + DOWN.dy)
    
    def trace_path(self, newI, newJ, i, j):
        self.trace[newI][newJ] = (i, j)

    def get_paths(self):
        stack = deque()
        i, j = self.get_destination()
        
        stack.appendleft((i, j))

        while not self.is_start(i, j):
            i, j = self.trace[i][j]
            stack.appendleft((i, j))
        
        return list(stack)

In [6]:
from queue import Queue


class Solver:
    def __init__(self, board):
        self.board = board
    
    def solve(self):
        queue = Queue()
        visited = set()
        is_arrived = False
        
        queue.put(self.board.get_start())
        
        while not queue.empty():
            i, j = queue.get()
            
            if self.board.is_destination(i, j):
                is_arrived = True
                break
            
            for newI, newJ in self.board.neighbors(i, j):
                if (newI, newJ) not in visited and not self.board.is_blocked(newI, newJ):
                    visited.add((newI, newJ))
                    queue.put((newI, newJ))
                    self.board.trace_path(newI, newJ, i, j)
        
        if not is_arrived:
            return None

        return self.board.get_paths()

In [7]:
class SolverRec:
    def __init__(self, board):
        self.board = board
    
    def solve(self):
        hash_table = {'result': None}

        def recur(result, i, j):
            if self.board.is_destination(i, j):
                result += [(i, j)]
                hash_table['result'] = result
                raise Error()
            else:
                for newI, newJ in self.board.neighbors(i, j):
                    if not self.board.is_blocked(newI, newJ):
                        recur(result + [(i, j)], newI, newJ)

        i, j = self.board.get_start()

        try:
            recur([], i, j)
        except:
            pass
        
        return hash_table['result']

In [8]:
solver = Solver(
    Board([
        [0, 0, 0, 0, 0],
        [0, 1, 0, 1, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]
    ])
)

print(solver.solve())

solver = SolverRec(
    Board([
        [0, 0, 0, 0, 0],
        [0, 1, 0, 1, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]
    ])
)

print(solver.solve())

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (3, 3), (3, 4)]
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (3, 3), (3, 4)]


8.3 Magic Index

In [9]:
def find_magic_index(ls):
    def recur(left, mid, right):
        if left <= right:
            if ls[mid] == mid:
                return ls[mid]
            elif ls[mid] > mid:
                return recur(left, left + (mid - 1 - left) // 2, mid - 1)
            else:
                return recur(mid + 1, mid + 1 + (right - mid - 1) // 2, right)

    
    length = len(ls)
    return recur(0, (length - 1) // 2, length - 1)

In [10]:
find_magic_index([-2, 0, 2, 4, 8, 10, 16, 18])

2

8.4 Power Set

In [11]:
def power_set(elements):
    def recur(step):
        if step == 0:
            yield frozenset()
        else:
            for current_set in recur(step - 1):
                yield current_set
                yield current_set | frozenset([elements[step - 1]])

    return recur(len(elements))

In [12]:
list(power_set([1, 2, 3]))

[frozenset(),
 frozenset({3}),
 frozenset({2}),
 frozenset({2, 3}),
 frozenset({1}),
 frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 2, 3})]

8.5 Recursive Multiply

In [13]:
def recursive_multiply(a, b):
    def recur(a, b):
        if a == 0 or b == 0:
            return 0
        
        if a < b:
            return b + recur(a - 1, b)
        else:
            return a + recur(a, b - 1)
    
    sign = -1 if (a > 0 and b < 0) or (a < 0 and b > 0) else 1
    return sign * recur(abs(a), abs(b))

In [14]:
recursive_multiply(4, 100)

400

8.6 Towers of Hanoi

In [15]:
def towers_of_hanoi(source, auxiliary, target):
    def recur(n, source, auxiliary, target):
        if n > 0:
            recur(n - 1, source, target, auxiliary)
            
            if source[1]:
                disk = source[1].pop()
                target[1].append(disk)
                
                print("Move {} from {}: {} to {}: {}".format(disk, source[0], source[1], target[0], target[1]))
            
            recur(n - 1, auxiliary, source, target)
    
    return recur(len(source[1]), source, target, auxiliary)

In [16]:
source = ("Source", [4,3,2,1])
target = ("Target", [])
auxiliary = ("Auxiliary", [])

towers_of_hanoi(source, auxiliary, target)

Move 1 from Source: [4, 3, 2] to Target: [1]
Move 2 from Source: [4, 3] to Auxiliary: [2]
Move 1 from Target: [] to Auxiliary: [2, 1]
Move 3 from Source: [4] to Target: [3]
Move 1 from Auxiliary: [2] to Source: [4, 1]
Move 2 from Auxiliary: [] to Target: [3, 2]
Move 1 from Source: [4] to Target: [3, 2, 1]
Move 4 from Source: [] to Auxiliary: [4]
Move 1 from Target: [3, 2] to Auxiliary: [4, 1]
Move 2 from Target: [3] to Source: [2]
Move 1 from Auxiliary: [4] to Source: [2, 1]
Move 3 from Target: [] to Auxiliary: [4, 3]
Move 1 from Source: [2] to Target: [1]
Move 2 from Source: [] to Auxiliary: [4, 3, 2]
Move 1 from Target: [] to Auxiliary: [4, 3, 2, 1]


8.7 Permutations without Dups

In [17]:
def permutations(ls):
    def recur(ls, result):
        if not ls:
            yield result
        else:
            for index in range(len(ls)):
                yield from recur(ls[:index] + ls[index + 1:], result + [ls[index]])
    return recur(ls, [])

In [18]:
list(permutations(['a', 'b', 'c']))

[['a', 'b', 'c'],
 ['a', 'c', 'b'],
 ['b', 'a', 'c'],
 ['b', 'c', 'a'],
 ['c', 'a', 'b'],
 ['c', 'b', 'a']]

8.8 Permutations with Dups

In [19]:
def permutations_with_dups(ls):
    result = set()

    def recur(ls, acc):
        if not ls:
            result.add(acc)
        else:
            for index in range(len(ls)):
                recur(ls[:index] + ls[index + 1:], acc + (ls[index],))

    recur(ls, tuple())
    
    return result

In [20]:
permutations_with_dups(['a', 'a', 'c'])

{('a', 'a', 'c'), ('a', 'c', 'a'), ('c', 'a', 'a')}

8.9 Parens

In [21]:
def parens(n):
    if n < 1:
        raise ValueError()
    
    def recur(acc):
        if acc == 1:
            yield "()"
        else:
            for paren in recur(acc - 1):
                yield "(){}".format(paren)
                yield "{}()".format(paren)
                yield "({})".format(paren)
                
    # can we not use set in this case?
    return set(recur(n))

In [22]:
def parens_left_right(n):
    if n < 1:
        raise ValueError()
    
    num = n * 2
    result = [None] * num
    
    def recur(left, right):
        if left == 0 and right == 0:
            print(''.join(result))
            return

        pos = num - (left + right)
        
        if left > 0:
            result[pos] = '('
            recur(left - 1, right)
        
        if left < right:
            result[pos] = ')'
            recur(left, right - 1)
    
    recur(n, n)

In [23]:
parens(3)

{'((()))', '(()())', '(())()', '()(())', '()()()'}

In [24]:
parens_left_right(3)

((()))
(()())
(())()
()(())
()()()


In [25]:
def floodFill(image, sr, sc, newColor):
    R, C = len(image), len(image[0])
    color = image[sr][sc]
    if color == newColor: return image
    def dfs(r, c):
        if image[r][c] == color:
            image[r][c] = newColor
            if r >= 1: dfs(r-1, c)
            if r+1 < R: dfs(r+1, c)
            if c >= 1: dfs(r, c-1)
            if c+1 < C: dfs(r, c+1)

    dfs(sr, sc)
    return image

In [26]:
image = [[1,1,1],[1,1,0],[1,0,1]]

In [27]:
floodFill(image, sr=1, sc=1, newColor=2)

[[2, 2, 2], [2, 2, 0], [2, 0, 1]]

8.11 Coins

In [28]:
def coins(ls, n):
    def recur(ls, n, count):
        if n < 0:
            return count
        
        if not ls:
            return count + 1 if n == 0 else count
        
        return recur(ls[1:], n, count) + recur(ls, n - ls[0], count)
    
    return recur(ls, n, 0)

In [29]:
def coins_dp(ls, n):
    dp = [[1 if column == 0 else 0 for column in range(n + 1)] for row in range(len(ls) + 1)]
    
    for index, coin in enumerate(ls):
        for money in range(n + 1):
            dp[index + 1][money] = dp[index][money]
            
            if money - coin >= 0:
                dp[index + 1][money] += dp[index + 1][money - coin]
    
    return dp[len(ls)][n]

In [30]:
print(coins([25, 10, 5, 1], 6))
print(coins_dp([25, 10, 5, 1], 6))

2
2


8.12 Eight Queens

In [31]:
from collections import deque


def is_safe(column, qs):
    row = len(qs)
    queens_with_rows = zip(range(row - 1, -1, -1), qs)
    
    return all([column != c and abs(column - c) != abs(row - r) for r, c in queens_with_rows])
    
def queens(n):
    def place_queens(k):
        if k == 0:
            yield []
        else:
            for qs in place_queens(k - 1):
                for column in range(n):
                    if is_safe(column, qs):
                        yield [column] + qs

    return place_queens(n)

In [32]:
for result in queens(4):
    for j in result:
        for k in range(4):
            if k == j:
                print('x', end=' ')
            else:
                print('o', end=' ')
        print()
    print()

o o x o 
x o o o 
o o o x 
o x o o 

o x o o 
o o o x 
x o o o 
o o x o 



8.13 Stack of Boxes

In [33]:
def longest_increasing_seqs(ls):
    length = len(ls)
    d = [0] * length
    trace = list(range(len(ls)))
    
    for i in range(length - 1, -1, -1):
        best_choice = i

        for j in range(i + 1, length):
            if d[best_choice] < d[j] and ls[i] <= ls[j]:
                best_choice = j
        
        d[i] = d[best_choice] + 1
        trace[i] = best_choice

    pointer = d.index(max(d))
    path = [ls[pointer]]
    
    while pointer != trace[pointer]:
        pointer = trace[pointer]
        path.append(ls[pointer])

    return path

In [34]:
longest_increasing_seqs([12, 9, 10, 22, 9, 33, 21, 50, 41, 60, 80, 34])

[9, 10, 22, 33, 50, 60, 80]

In [35]:
longest_increasing_seqs([1, 3, 2, 3, 4, 8, 7, 9])

[1, 3, 3, 4, 8, 9]

8.14 Boolean Evaluation

In [36]:
def parseBoolExpr(expression: str) -> bool:
    def parse(e: str, lo: int, hi: int) -> bool:
        if hi - lo == 1: # base case
            return e[lo] == 't'               
        ans = e[lo] == '&' # only when the first char is '&', ans assigned True.
        level, start = 0, lo + 2 # e[lo + 1] must be '(', so start from lo + 2 to delimit sub-expression.
        for i in range(lo + 2, hi):
            if level == 0  and e[i] in [',', ')']: # found a sub-expression.
                cur = parse(e, start, i) # recurse to sub-problem.
                start = i + 1 # start point of next sub-expression.
                if e[lo] == '&':
                    ans &= cur
                elif e[lo] == '|':
                    ans |= cur
                else: # e[lo] is '!'.
                    ans = not cur
            if e[i] == '(':
                level = level + 1
            elif e[i] == ')':
                level = level - 1
        return ans;        

    return parse(expression, 0, len(expression))

def parseBoolExpr1(expression: str) -> bool:
    stack = []
    for c in expression:
        if c == ')':
            seen = set()
            while stack[-1] != '(':
                seen.add(stack.pop())
            stack.pop()
            operator = stack.pop()
            stack.append(all(seen) if operator == '&' else any(seen) if operator == '|' else not seen.pop())
        elif c != ',':
            stack.append(True if c == 't' else False if c == 'f' else c)
    return stack.pop()

In [37]:
expression = "!(f)"
print(parseBoolExpr(expression))
print(parseBoolExpr1(expression))

True
True
